## 디코더의 두번째 서브층: 인코더-디코더 어텐션

**참고문헌**  
\[1\] [16-01 트랜스포머(Transformer)](https://wikidocs.net/31379)  
\[2\] [The Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html#prelims)  

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

In [2]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])
    
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

#batch: number of batch
#nbatches : nuber of batch run.
#V : num of vacabulary
def data_gen(V, batch, nbatches, seq_len=10):
    "Generate random data for a src-tgt copy task."
    for i in range(nbatches):
        data = torch.from_numpy(np.random.randint(1, V, size=(batch, seq_len)))
        #시작 토큰(Start Token)을 고정하는 역할
        data[:, 0] = 1
        src = Variable(data, requires_grad=False)
        tgt = Variable(data, requires_grad=False)
        yield Batch(src, tgt, 0)
    
class Batch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, src, trg=None, pad=0):
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.trg_mask = \
                self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).data.sum()
    
    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        #print('tgt_mask.shape', tgt_mask.shape)
        #print('tgt_mask', tgt_mask)
        tgt_mask = tgt_mask & Variable(
            subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        return tgt_mask

class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model
    #x.shape = (batch_size, sequence_len)
    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model) 

![fig1](../images/decoder._look_a_head.png)

디코더의 두번째 서브층에 대해서 이해해봅시다. 디코더의 두번째 서브층은 멀티 헤드 어텐션을 수행한다는  
점에서는 이전의 어텐션들(인코더와 디코더의 첫번째 서브층)과는 공통점이 있으나 이번에는 셀프 어텐션이 아님.  
<br>
셀프 어텐션은 Query, Key, Value가 같은 경우를 말하는데, 인코더-디코더 어텐션은 Query가 디코더인  
행렬인 반면, Key와 Value는 인코더 행렬이기 때문입니다. 다시 한 번 각 서브층에서의 Q,K,V의 관계를 정리해봄.  

|    layer             |                       |         
|----------------------|------------------------|
|인터더의 첫번째 서브층  | Query = **Key** = Value |
|디코더의 첫번째 서브층  | Query = **Key** = Value |
|디코더의 두번째 서브층  | Query : 디코더 행렬 /**Key = Value** : 인코더 행렬 |

디코더의 두번째 서브층을 확대해 보면, 다음과 같이 인코더로부터 두 개의 화살표가 그려져 있음.  

![fig_3](../images/decoder_2nd_layer.png)

두 개의 화살표는 각각  key와 value를 의미하며 이는 인코더의 마지막 층에서 온 행렬로부터 얻음.  
반면 Query는 더코더의 첫번째 서브층의 결과 행렬로부터 얻는다는 점이 다릅니다.  
Query가 디코더 행렬 Key가 인코더 행렬일 때, 어텐셜 스코어 행렬을 추구하는 과정은 다음과 같습니다.

![fig_4](../images/decoder_2nd_attention_score_matrix_final.png)

In [6]:
## vector : row major 
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    print("scores.shape : ", scores.shape)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    print("value.shape : ", value.shape)
    print("p_attn.shape : ", p_attn.shape)
    return torch.matmul(p_attn, value), p_attn

class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]

        print("query.shape ", query.shape)
        print("key.shape ", key.shape)
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)



2nd layer의 attention layer에 들어 q,k,v값의 차원이 달라도  
결과는 q의 차원으로 나옴.
> 예  
q = (1,9,512)  
k = (1,10, 512)  
v = (1,10, 512)  
일 때  
출력은 y =(1,9,512) 로 나옴

In [5]:
V=10
batch = 1
d_model = 512
h=8
embedding = Embeddings(d_model, V)
data = torch.from_numpy(np.random.randint(1, V, size=(batch, 10)))
#시작 토큰(Start Token)을 고정하는 역할
data[:, 0] = 1
src = Variable(data, requires_grad=False)
tgt = Variable(data, requires_grad=False)
batch = Batch(src, tgt, 0)
m = embedding(batch.src)
t = embedding(batch.trg)

print("t.shape : ", t.shape)
print("m.shape : ", m.shape)
att = MultiHeadedAttention(h,d_model)

y = att(t,m,m,batch.src_mask)

print("y.shape : ", y.shape)

t.shape :  torch.Size([1, 9, 512])
m.shape :  torch.Size([1, 10, 512])
query.shape  torch.Size([1, 8, 9, 64])
key.shape  torch.Size([1, 8, 10, 64])
scores.shape :  torch.Size([1, 8, 9, 10])
value.shape :  torch.Size([1, 8, 10, 64])
p_attn.shape :  torch.Size([1, 8, 9, 10])
y.shape :  torch.Size([1, 9, 512])
